In [ ]:
import arcpy
import os
arcpy.env.overwriteOutput = True

In [ ]:
svi = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\data\svi/svi_tx.shp'
evac = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\data\evac_routes/TxDOT_Evacuation_Routes.shp'
roads = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\data\roadways/TxDOT_Roadways.shp'
hwm = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\data\high_water_marks\USGS_Harvey_HWM_Peaks/USGS_Harvey_HWM_Peaks.shp'
damage = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\data\building_damage/building_damage.shp'

In [ ]:
# roads = 'TxDOT_Roadways'
# evac = 'TxDOT_Evacuation_Routes'
# svi = 'svi_tx'
# hwm = 'USGS_Harvey_HWM_Peaks'
# damage = 'building_damage'
roads_clip = 'roads_clipped'
svi_cent = 'svi_centroids'
intersect = 'intersection_points'
clipped_svi_hwm = 'svi_clipped_hwm'
clipped_svi_damage = 'svi_clipped_damage'
hwm_clip = 'hwm_clipped'
damage_clip = 'damage_clipped'

In [ ]:
## Check input feature class spatial reference
roads_desc = arcpy.Describe(roads)
roads_desc_name = roads_desc.spatialReference.name
svi_desc = arcpy.Describe(svi)
svi_desc_name = svi_desc.spatialReference.name
evac_desc = arcpy.Describe(evac)
evac_desc_name = evac_desc.spatialReference.name
hwm_desc = arcpy.Describe(hwm)
hwm_desc_name = hwm_desc.spatialReference.name
damage_desc = arcpy.Describe(damage)
damage_desc_name = damage_desc.spatialReference.name

In [ ]:
## Ensure projections all match, using roads as the defining piece. Will convert SVI to WGS-84 at the end for
## for compatability with Geographic Visualization project. 

if roads_desc_name != svi_desc_name:
    print("Roads and Evac do not match, updating svi in svi_reprojected to match roads: " + roads_desc_name)
    arcpy.management.Project(svi, 'svi_reprojected', roads_desc.spatialReference)
    inFacilities = arcpy.env.workspace + '/inFacilities_reprojected'
    print("Projection updated, both svi and roads are projected in: " + roads_desc_name)
else:
    print("SVI feature class spatial reference matches.")
    
if roads_desc_name != evac_desc_name:
    print("Roads and Evac do not match, updating evac in evac_reprojected to match roads: " + roads_desc_name)
    arcpy.management.Project(evac, 'evac_reprojected', roads_desc.spatialReference)
    evac = arcpy.env.workspace + '/evac_reprojected'
    print("Projection updated, both evac and roads are projected in: " + roads_desc_name)
else:
    print("Evac feature class spatial reference matches.")
    
if roads_desc_name != hwm_desc_name:
    print("Roads and HWM do not match, updating HWM in hwm_reprojected to match roads: " + roads_desc_name)
    arcpy.management.Project(hwm, 'hwm_reprojected', roads_desc.spatialReference)
    hwm = arcpy.env.workspace + '/hwm_reprojected'
    print("Projection updated, both hwm and roads are projected in: " + roads_desc_name)
else:
    print("High Water Marks feature class spatial reference matches.")
    
if roads_desc_name != damage_desc_name:
    print("Roads and damage do not match, updating damage in damage_reprojected to match roads: " + roads_desc_name)
    arcpy.management.Project(damage, 'damage_reprojected', roads_desc.spatialReference)
    hwm = arcpy.env.workspace + '/damage_reprojected'
    print("Projection updated, both damage and roads are projected in: " + roads_desc_name)
else:
    print("Damage feature class spatial reference matches.")

In [ ]:
## clip high water marks to be within the svi region (Must be done first, as the svi is clipped after using these points)
arcpy.analysis.Clip(hwm, svi, hwm_clip)

In [ ]:
## spatial join to clip svi down to hwm areas
arcpy.analysis.SpatialJoin(svi, hwm_clip, clipped_svi_hwm, join_type = 'KEEP_COMMON',
                           match_option = 'INTERSECT')

In [ ]:
## Convert census block groups to centroids for point to point drive time analysis.
arcpy.management.FeatureToPoint(clipped_svi_hwm, svi_cent, "CENTROID")

In [ ]:
## intersect analysis between roads and evacuation routes
intersect_features = [roads, evac]
arcpy.analysis.Intersect(intersect_features, intersect, 'ONLY_FID', output_type = 'POINT')

## Near 

In [ ]:
arcpy.analysis.Near(svi_cent, intersect, method = "GEODESIC")

## Statistics

In [ ]:
hwm_summary = 'hwm_summary'
damage_summary = 'damage_summary'

In [ ]:
arcpy.analysis.SummarizeWithin(clipped_svi_hwm, hwm_clip, hwm_summary, 'KEEP_ALL', [['height_abo', 'MEAN']])

## Join

In [ ]:
joinField = 'FIPS'

In [ ]:
arcpy.management.AddJoin(hwm_summary, joinField, svi_cent, joinField)

## New Table with New Fields 

In [ ]:
in_fc = 'hwm_summary'
out_path = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\outputs'
out_fc = 'summary_output'
keep_fields = ['FIPS', 'EP_POV', 'EP_UNEMP', 'EP_MUNIT', 'EP_MOBILE', 'EP_NOVEH', 'EP_UNINSUR']

In [ ]:
## achieved by manually running the feature class to feature class conversion tool and copying the Field Map output from the 'results' tab. Must be performed again if desired fields or table names are different.
fieldMappings = """FIPS "FIPS" true true false 254 Text 0 0,First,#,hwm_summary,hwm_summary.FIPS,0,254;EP_POV "EP_POV" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.EP_POV,-1,-1;EP_UNEMP "EP_UNEMP" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.EP_UNEMP,-1,-1;EP_MUNIT "EP_MUNIT" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.EP_MUNIT,-1,-1;EP_MOBILE "EP_MOBILE" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.EP_MOBILE,-1,-1;EP_NOVEH "EP_NOVEH" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.EP_NOVEH,-1,-1;RPL_THEMES "RPL_THEMES" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.RPL_THEMES,-1,-1;E_UNINSUR "E_UNINSUR" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.E_UNINSUR,-1,-1;MEAN_height_abo "Average height_abo" true true false 8 Double 0 0,First,#,hwm_summary,hwm_summary.MEAN_height_abo,-1,-1;NEAR_DIST "NEAR_DIST" true true false 8 Double 0 0,First,#,hwm_summary,svi_centroids.NEAR_DIST,-1,-1"""

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion(in_fc, out_path, out_fc, field_mapping = fieldMappings)

In [ ]:
output_folder = r'D:\Documents\GitHub Repos\geospatial_programming_assignments\Final Project\Final Project\outputs'
arcpy.conversion.FeatureClassToShapefile(out_fc, output_folder)

## Shit to integrate damage and high water mark points and do a weighted average (not working)

In [ ]:
## clip damage to be within the svi region (Must be done first, as the svi is clipped after using these points)
# arcpy.analysis.Clip(damage, svi, damage_clip)

In [ ]:
# ## spatial join to clip svi down to damage areas
# arcpy.analysis.SpatialJoin(svi, damage_clip, clipped_svi_damage, join_type = 'KEEP_COMMON',
#                            match_option = 'INTERSECT')

In [ ]:
mergeOut = 'merged_svi_areas'

In [ ]:
# arcpy.management.Merge([clipped_svi_hwm, clipped_svi_damage], mergeOut)

In [ ]:
# arcpy.management.DeleteIdentical(mergeOut, 'FIPS')

In [ ]:
# arcpy.analysis.SummarizeWithin(mergeOut, damage_clip, damage_summary, 'KEEP_ALL', [['IN_DEPTH', 'MEAN']])

In [ ]:
# arcpy.management.AddJoin(hwm_summary, joinField, damage_summary, joinField)

In [ ]:
# newField = 'wa_depth'

In [ ]:
## Calculate weighted average innundation depth
# arcpy.management.AddField(hwm_summary, newField, 'DOUBLE')

In [ ]:
# expression = "((!damage_summary.Point_Count! * !damage_summary.MEAN_IN_DEPTH!) + (!hwm_summary.Point_Count! * !MEAN_height_abo!)) / (!damage_summary.Point_Count! + !hwm_summary.Point_Count!)"
# expression

In [ ]:
# arcpy.CalculateField_management(hwm_summary, newField, expression, "PYTHON3")

In [ ]:
# finalTable = 'final_table'
# arcpy.management.CopyFeatures(hwm_summary, finalTable)

## Network Bullshit

In [ ]:
arcpy.CheckOutExtension("network")

In [ ]:
## Create featuredataset for roads network to calculate driving distance.
arcpy.CreateFeatureDataset_management(arcpy.env.workspace, 'featuredataset', roads_desc.spatialReference)

In [ ]:
## copy cleaned features into featuredataset
arcpy.CopyFeatures_management(roads_clip, 'featuredataset/roads_network')
arcpy.CopyFeatures_management(svi_cent, 'featuredataset/svi_centroids_network')
arcpy.CopyFeatures_management(intersect, 'featuredataset/intersect_network')

In [ ]:
# ## create roads network
# arcpy.na.CreateNetworkDataset('featuredataset', 'roads_ND', ['roads'])
arcpy.BuildNetwork_na('featuredataset/roads_ND')

In [ ]:
travel_modes = arcpy.na.GetTravelModes('featuredataset/roads_ND')
for travel_mode_name in travel_modes:
    travel_mode = travel_modes[travel_mode_name]
    print(travel_mode_name)
    print("Impedance:", travel_mode.impedance)
    print("Restrictions:", ", ".join(travel_mode.restrictions))
    print("")

In [ ]:
travel_modes

In [ ]:
travel_mode = arcpy.na.TravelMode(travel_modes["Length"])

In [ ]:
## network layer variables
outRoutes = 'closest_facility_layer'
timeOfDay = '8/26/2017 12:00 PM'

In [ ]:
## create closest facility calculation layer
result_object = arcpy.na.MakeClosestFacilityAnalysisLayer('roads_ND', outRoutes, "Driving Time", 'TO_FACILITIES',
                                          time_of_day = timeOfDay, time_zone = 'LOCAL_TIME_AT_LOCATIONS', 
                                          time_of_day_usage = 'START_TIME', line_shape = 'ALONG_NETWORK')
#                                            accumulate_attributes = ["Meters", "Travel Time"])

In [ ]:
## layer object from the result, allows us to reference the layer
layer_object = result_object.getOutput(0)

In [ ]:
#Get the names of all the sublayers within the closest facility layer.
sublayer_names = arcpy.na.GetNAClassNames(layer_object)
#Stores the layer names that we will use later
facilities_layer_name = sublayer_names["Facilities"]
incidents_layer_name = sublayer_names["Incidents"]

In [ ]:
#Load the intersections as Facilities using the default field mappings and
#search tolerance
arcpy.na.AddLocations(layer_object, facilities_layer_name,
                        'featuredataset/intersect', "", "500000 METERS")

In [ ]:
#Load the centroids as Incidents. Map the Name property from the NOM field
#using field mappings
field_mappings = arcpy.na.NAClassFieldMappings(layer_object,
                                                incidents_layer_name)
field_mappings["Name"].mappedFieldName = "NOM"
arcpy.na.AddLocations(layer_object, incidents_layer_name, 'featuredataset/svi_cents',
                      field_mappings, "")

In [ ]:
## solve - find the quickest route from census block group centroids to highway and road intersections (resresent on ramps)
arcpy.na.Solve(layer_object)